# 1) Imports

In [131]:
import pandas as pd
import numpy as np
import unidecode
import re

# 2) Coleta dos dados

## 2.1) Municípios potenciais

In [174]:
top_muns = pd.read_csv("../Dados/Tabela_final/municipios_potenciais.csv").drop(columns=['Unnamed: 0'])

## 2.2) Dados de produção agrícola

### 2.2.1) Extração e transformação

Os arquivos .csv dos dados de produção agrícola municipal apresentavam uma estrutura conflitante de forma que tiveram que ser editados manualmente. Para resolver o conflito, deve-se abrir manualmente os arquivos .csv da pasta PAM e excluir as linhas auxiliarem antes e depois dos dados.

In [164]:
def limpar_nome(nome):
    nome_sem_acentos = unidecode.unidecode(nome)  # Remove acentos
    nome_sem_codigo_estado = re.sub(r'\(\w+\)', '', nome_sem_acentos)  # Remove a sequência "(XX)"
    return nome_sem_codigo_estado.upper()  # Converte para maiúsculas

def limpar_dataframe(df, valor):
    
    municipios = df['Brasil e Município'].apply(limpar_nome)
    new_df = pd.DataFrame()
    for i in range(0,3):
        new_year_df = df.iloc[:, 1 + 37 * i : 37 * (i + 1)]
        new_year_df.columns = [column.rstrip('.1').rstrip('.2') for column in new_year_df.columns]
        new_year_df.insert(0, 'MUNICIPIO', list(municipios))
        new_df = pd.concat([new_df, new_year_df], ignore_index = True)
    new_df = new_df.replace(['-', '...'], 0)
    new_df = new_df.fillna(0)
    new_df = new_df.melt(id_vars=['MUNICIPIO'], var_name='PRODUTO', value_name=valor)
    new_df[valor] = new_df[valor].astype('int64')
    new_df = new_df[['MUNICIPIO', 'PRODUTO', valor]].groupby(['MUNICIPIO', 'PRODUTO']).mean().reset_index()
    new_df['PRODUTO'] = new_df['PRODUTO'].apply(limpar_nome)

    return new_df

area_colhida = pd.read_csv("../Dados/PAM/pam_area_colhida.csv",sep=';')
area_plantada = pd.read_csv("../Dados/PAM/pam_area_plantada.csv",sep=';')
rend_medio = pd.read_csv("../Dados/PAM/pam_rendimento_medio.csv",sep=';')
valor_producao = pd.read_csv("../Dados/PAM/pam_valor_producao.csv",sep=';')

area_colhida = limpar_dataframe(area_colhida, 'AREA_COLHIDA')
area_plantada = limpar_dataframe(area_plantada, 'AREA_PLANTADA')
rend_medio = limpar_dataframe(rend_medio, 'REND_MEDIO')
valor_producao = limpar_dataframe(valor_producao, 'VALOR_PROD')

agro = area_plantada.merge(area_colhida, on = ['MUNICIPIO', 'PRODUTO'], how = 'inner')
agro = agro.merge(rend_medio, on = ['MUNICIPIO', 'PRODUTO'], how = 'inner')
agro = agro.merge(valor_producao, on = ['MUNICIPIO', 'PRODUTO'], how = 'inner')
agro['MUNICIPIO'] = agro['MUNICIPIO'].str.strip()
agro= agro[(agro['AREA_PLANTADA']>0)&(agro['AREA_COLHIDA']>0)&(agro['REND_MEDIO']>0)&(agro['VALOR_PROD']>0)]
agro.to_csv("../Dados/Tabela_final/dados_producao_agricola.csv")
agro

,MUNICIPIO,PRODUTO,AREA_PLANTADA,AREA_COLHIDA,REND_MEDIO,VALOR_PROD
0,ABAIARA,ALFAFA FENADA,0.000000,0.000000,0.000000,0.000000
1,ABAIARA,ALGODAO ARBOREO (EM CAROCO),0.000000,0.000000,0.000000,0.000000
2,ABAIARA,ALGODAO HERBACEO (EM CAROCO),26.666667,26.666667,2796.333333,214.666667
3,ABAIARA,AMENDOIM (EM CASCA),15.000000,15.000000,978.000000,60.666667
4,ABAIARA,BANANA,26.666667,26.666667,18363.333333,786.333333
...,...,...,...,...,...,...
51907,ZABELE,TRIGO (EM GRAO),0.000000,0.000000,0.000000,0.000000
51908,ZABELE,TRITICALE (EM GRAO),0.000000,0.000000,0.000000,0.000000
51909,ZABELE,TUNGUE (FRUTO SECO),0.000000,0.000000,0.000000,0.000000
51910,ZABELE,URUCUM,0.000000,0.000000,0.000000,0.000000


### 2.2.1) Carregamento

In [177]:
agro = pd.read_csv("../Dados/Tabela_final/dados_producao_agricola.csv")